<a href="https://colab.research.google.com/github/teelch0/Data-Mining/blob/main/trainable_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention with trainable weights

In [ ]:
import torch

In [ ]:
inputs = torch.nn.Embedding( 4, 8 )

In [ ]:
inputs = inputs.weight
inputs

Parameter containing:
tensor([[-0.0674,  0.0207, -0.5329, -0.8729,  1.1554, -0.9480,  1.5168,  0.7848],
        [ 0.5720, -1.8700,  0.8103,  0.5913,  0.2984, -0.8857,  0.1027,  1.2259],
        [ 0.9494, -0.7322, -1.0040, -0.5465, -0.0665, -0.5673,  0.4528,  0.3364],
        [-1.2101, -0.0706, -1.4929,  1.3864, -0.3227, -0.3523, -0.1533,  1.0407]],
       requires_grad=True)

In [ ]:
inputs = inputs.data
inputs

tensor([[-0.0674,  0.0207, -0.5329, -0.8729,  1.1554, -0.9480,  1.5168,  0.7848],
        [ 0.5720, -1.8700,  0.8103,  0.5913,  0.2984, -0.8857,  0.1027,  1.2259],
        [ 0.9494, -0.7322, -1.0040, -0.5465, -0.0665, -0.5673,  0.4528,  0.3364],
        [-1.2101, -0.0706, -1.4929,  1.3864, -0.3227, -0.3523, -0.1533,  1.0407]])

In [ ]:
# set dimensions
d_in = 8
d_out = 6

# create weight matrices
W_q = torch.nn.Parameter( torch.rand( d_in, d_out ), requires_grad=False )
W_k = torch.nn.Parameter( torch.rand( d_in, d_out ), requires_grad=False )
W_v = torch.nn.Parameter( torch.rand( d_in, d_out ), requires_grad=False )

In [ ]:
# choose an input vector and transform it into our query vector using W_q
query = inputs[2] @ W_q
query

tensor([ 1.0332,  0.0513, -0.8901, -1.3339,  0.2399,  0.0196])

In [ ]:
# calculate attention scores using the keys generated by W_k:
keys = inputs @ W_k
values = inputs @ W_v
print("Keys:", keys)
print("Values:", values )

Keys: tensor([[ 1.8223, -0.5644, -0.6595,  0.0935,  0.9502, -0.6515],
        [ 1.3030,  1.3933,  0.0372,  0.3829, -0.3306, -1.2702],
        [ 0.3867, -0.8524, -0.8998, -0.9065, -1.5789, -1.7856],
        [-0.6392, -1.0941, -1.4458, -0.0668, -0.6042, -1.1222]])
Values: tensor([[-0.4964,  0.3235,  0.5167,  1.1451,  1.2204,  0.8450],
        [ 1.4305,  1.4904,  0.3068,  0.1569,  1.5575,  0.1329],
        [-1.1040, -0.3154, -1.0121, -1.6176, -0.3838, -0.2960],
        [-0.5260,  0.5710, -0.8826, -0.2251, -1.5533, -0.2848]])


In [ ]:
attention_scores = query @ keys.T
attention_scores

tensor([2.5315, 0.7696, 1.9521, 0.4925])

In [ ]:
attention_weights = torch.softmax( attention_scores, dim = -1 )
attention_weights

tensor([0.5370, 0.0922, 0.3009, 0.0699])

In [ ]:
attention_weights.sum()

tensor(1.)

In [ ]:
context_vector = attention_weights @ values
context_vector

tensor([-0.5036,  0.2562, -0.0604,  0.1270,  0.5750,  0.3571])